In [ ]:
import sys
sys.path.append('/Users/JIC115/Desktop/CBS_prj/Rose_PDO_model/')

from util.util_eval import * 

from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay,classification_report
from sklearn.preprocessing import label_binarize

import pandas as pd 
#pd.set_option('display.max_columns',None)
#pd.set_option('display.max_rows',None)

wd = Path('/Users/JIC115/Desktop/CBS_prj/Rose_PDO_model/validation/data/')
wd1 = wd.joinpath('qprj_pred_all_PDO_wsi/')
wd2 = wd.joinpath('qprj_PDO_remove_train_cell/')
wd_fig = wd2.joinpath('roc_pdf')

# 1. Remove training cells from wsi

In [ ]:
offset_tile = {
    'HN23-10730-p4_H&E_1':[20094,11522],
    'HN23-10752-p1_H&E_1':[6490,7901],
    'HN23-10752-p2_H&E_1':[6680,6646],
    'HN24-10791-p1_H&E_1':[7977,5023],
    'HN24-10810-p1_H&E_1':[7711,6496],
    'HN24-10831-p3_H&E_1':[18826,10292],
    'HN24-10909-P2_H&E': [0,0]}

# see large_tile_coord_on_wsi

In [ ]:
# add tile offset
indir = wd2.joinpath('train_val_final_gson_ori_cell_in_tiledim')
add_tile_offset(indir,offset_tile)

# convert train final gsons to wsi space
outdir = wd2.joinpath('train_val_final_gson_cell_in_wsidim')
gs_indir = indir
tileGs2wsiGs(outdir,gs_indir,tile_size=256,add_edge=True)

In [ ]:
tbl_fn = wd1.joinpath('pred_sunny_tables/Cell-level_train_pure.csv')
df = pd.read_csv(str(tbl_fn))
print(len(df))

# remove background
df = df.query("pred_type != 'background' and true_type != 'background'")
print(df.pred_type.unique())
print(df.true_type.unique())

####### make plots
ytrue = df.true_type 
ypred = df.pred_type

#-------- confusion plot
conf_mat = confusion_matrix(ytrue, ypred) # sorted by alphabeta order of labels !!!
label = sorted(['normal','tumor']) 
cmat_disp = ConfusionMatrixDisplay(conf_mat,display_labels=label)

# customize :
fig, ax = plt.subplots(figsize=(6.5,6.6))    # set plot figsize
cmat_disp.plot(ax=ax, colorbar=False)    # Deactivate default colorbar,size does not match with plot

# Adding custom colorbar
cax = fig.add_axes([ax.get_position().x1+0.01,ax.get_position().y0,0.02,ax.get_position().height])
plt.colorbar(cmat_disp.im_,  cax=cax)
plt.savefig(wd_fig.joinpath('Cell-level_train_pure_conf.pdf'))

#-------- ROC plot
yprob = df.prob_tumor
RocCurveDisplay.from_predictions(ytrue,yprob,pos_label='tumor')
plt.savefig(wd_fig.joinpath('Cell-level_train_pure_roc.pdf'))

#-------- report table
report = pd.DataFrame(classification_report(ytrue, ypred, output_dict = True))
report.rename(index = {'recall':'sensitivity'},inplace=True)
report
# plt.savefig(wd_fig.joinpath('Cell-level_train_pure_report.pdf'))

# 3. Remove PDOs whose cells were in training tiles

In [ ]:
### get report
tbl_fn = wd1.joinpath('pred_sunny_tables/PDO-level_train_pure.csv')
df = pd.read_csv(str(tbl_fn),index_col=0)
print(len(df)) # 
ytrue = df['ann_ori_type']
ypred = df['ann_pred_type']

#-------- confusion plot
conf_mat = confusion_matrix(ytrue, ypred) # sorted by alphabeta order of labels !!!
label = sorted(['normal','tumor']) 
cmat_disp = ConfusionMatrixDisplay(conf_mat,display_labels=label)

# customize :
fig, ax = plt.subplots(figsize=(6.5,6.6))    # set plot figsize
cmat_disp.plot(ax=ax, colorbar=False)    # Deactivate default colorbar,size does not match with plot

# Adding custom colorbar
cax = fig.add_axes([ax.get_position().x1+0.01,ax.get_position().y0,0.02,ax.get_position().height])
plt.colorbar(cmat_disp.im_,  cax=cax)
plt.savefig(wd_fig.joinpath('PDO-level_train_pure_conf.pdf'))

#-------- ROC plot
yprob = df.tumor_prob
RocCurveDisplay.from_predictions(ytrue,yprob,pos_label='tumor')
plt.savefig(wd_fig.joinpath('PDO-level_train_pure_roc.pdf'))

#-------- perf report
report = pd.DataFrame(classification_report(ytrue, ypred, output_dict = True))
report.rename(index = {'recall':'sensitivity'},inplace=True)
report